In [1]:
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import trade_functions as tf
import read_quote      as rq
import stock_functions as sf

import sys
import time

In [2]:
inpFile = 'quotes/ibm.csv'
myQuote = rq.readQuote( inpFile )

In [3]:
myQuote = myQuote.iloc[0:500].copy()
myQuote.is_copy=False
print myQuote.head()

             close     volume    open    high       low
date                                                   
2017/03/10  177.83  3100107.0  178.21  179.49  177.4200
2017/03/09  177.18  5443665.0  179.15  179.25  175.8800
2017/03/08  179.45  3557388.0  180.75  180.95  179.3000
2017/03/07  180.38  2977496.0  180.71  181.29  180.1997
2017/03/06  180.47  3199175.0  179.72  180.99  179.5700


In [23]:
# Daily percentage difference between close/open, rel to open  ( diff_co )
# Daily percentage difference between high/log, rel to open    ( diff_hl )
# Percentage change in volume between days                     ( diff_v  )

diffs = sf.generate_differentials( myQuote )
diffs.head()

,diff_co,diff_hl,diff_v
date,,,
2017/03/10,-0.002132,0.011616,-0.430511
2017/03/09,-0.010996,0.018811,0.530242
2017/03/08,-0.007192,0.009129,0.194758
2017/03/07,-0.001826,0.006033,-0.069293
2017/03/06,0.004173,0.007901,0.753058


In [25]:
# Rolling means   of closing values for number of days ( close_mean_<n_days> )
# Rolling stddevs of closing values for number of days ( close_std_<n_days>  )

rolls = sf.generate_rolling_close( myQuote, [ 3, 5, 10 ] )
rolls.head()

,close_mean_3,close_std_3,close_mean_5,close_std_5,close_mean_10,close_std_10
date,,,,,,
2017/03/10,178.153333,1.169031,179.062,1.494145,179.706,1.372064
2017/03/09,179.003333,1.646096,179.506,1.360379,180.058,1.286138
2017/03/08,180.100000,0.564712,180.176,0.446184,180.505,0.890746
2017/03/07,180.300000,0.221133,180.676,0.735921,180.675,0.826966
2017/03/06,180.350000,0.261534,180.564,0.828963,180.663,0.832574


In [26]:
# Percentage change in the stock price relative to a certain number of days in the past ( momentum_<n_days> )

moms = sf.generate_momentum_close( myQuote, [3,5,10] )
moms.head()

,momentum_3,momentum_5,momentum_10
date,,,
2017/03/10,-0.014137,-0.012330,-0.019410
2017/03/09,-0.018230,-0.018556,-0.024608
2017/03/08,-0.003332,-0.013740,-0.009384
2017/03/07,-0.000831,0.003114,0.000666
2017/03/06,-0.008134,0.005964,-0.001107


In [80]:
# Relative strength index normalized to 1, 1-1/(1+avg_gains_ndays/avg_loss_ndays)  (rsi_<n_days>)
# Oversold below ~30, overbought above ~70, buy low

rsi = sf.generate_rsi( myQuote, [10,20,30] )
rsi.head()

,rsi_10,rsi_20,rsi_30
date,,,
2017/03/10,0.336431,0.516903,0.486263
2017/03/09,0.285303,0.526962,0.481457
2017/03/08,0.401620,0.526400,0.559068
2017/03/07,0.506977,0.610676,0.637540
2017/03/06,0.488789,0.614138,0.644270


In [91]:
bands = sf.generate_bollinger_bands( myQuote, [5,10] )
bands.head()

,bollinger_5,bollinger_10
date,,
2017/03/10,0.293862,0.158179
2017/03/09,0.072545,-0.059427
2017/03/08,0.093217,0.203900
2017/03/07,0.399446,0.410819
2017/03/06,0.471651,0.442047


In [6]:
num1,num2,num3,num4,num5 = 3, 5, 10, 15, 20
num_list = [ num1, num2, num3, num4, num5 ]

In [33]:
for num in num_list:
    foo['avg_close_'+str(num)] = tf.calcRunAvg( foo, 'close', num )

In [36]:
%matplotlib inline
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = (14, 10)

In [39]:
%matplotlib

Using matplotlib backend: Qt4Agg


In [41]:
f  = plt.figure()
ax1 = f.add_subplot(211)

x  = np.arange( foo.shape[0], 0, -1 )
y1 = foo['close'].values
y2 = foo['avg_close_'+str(num1)].values
y3 = foo['avg_close_'+str(num2)].values
y4 = foo['avg_close_'+str(num3)].values
y5 = foo['avg_close_'+str(num4)].values
y6 = foo['avg_close_'+str(num5)].values

ax1.plot( x         , y1 , color = 'w' , linestyle='-', label='Close' )
ax1.plot( x-num1/2. , y2 , color = 'r' , linestyle='-', label='Avg %2i days'%num1 )
ax1.plot( x-num2/2. , y3 , color = 'b' , linestyle='-', label='Avg %2i days'%num2 )
ax1.plot( x-num3/2. , y4 , color = 'g' , linestyle='-', label='Avg %2i days'%num3 )
ax1.plot( x-num4/2. , y5 , color = 'c' , linestyle='-', label='Avg %2i days'%num4 )
ax1.plot( x-num5/2. , y6 , color = 'y' , linestyle='-', label='Avg %2i days'%num5 )

plt.legend( loc=4 )


ax2 = f.add_subplot(212,sharex=ax1)
#ax.plot( x                 , y1        /y1                 , color = 'w' , linestyle='-' )
ax2.plot( x[num1/2:-num1/2] , y2[:-num1]-y1[num1/2:-num1/2] , color = 'r' , linestyle='-' )
ax2.plot( x[num2/2:-num2/2] , y3[:-num2]-y1[num2/2:-num2/2] , color = 'b' , linestyle='-' )
ax2.plot( x[num3/2:-num3/2] , y4[:-num3]-y1[num3/2:-num3/2] , color = 'g' , linestyle='-' )
ax2.plot( x[num4/2:-num4/2] , y5[:-num4]-y1[num4/2:-num4/2] , color = 'c' , linestyle='-' )
ax2.plot( x[num5/2:-num5/2] , y6[:-num5]-y1[num5/2:-num5/2] , color = 'y' , linestyle='-' )


plt.show()
#plt.close()

In [95]:
# Covariance of all variables
# Break into chunks of ___ days, so average isn't skewed to general trend of rise in the market
# ^ Can use metrics to determine slumps, shooting up, constant growth, volatile
# Divide out running average, then generate covariance
# Gen covariance between days
# Plot trends over time
# Test differrent time varying models